In [26]:
import os
import lzma
import json
import pandas as pd
import numpy as np  # Import NumPy

# Define a list of folder paths here
folder_paths = [']

               
    # Add more folder paths as needed

In [61]:
import pandas as pd
import lzma
import os
import json
import numpy as np  # Import NumPy


folder_paths

def extract_fields_from_json(json_data):
    try:
        # Extract the specific fields you need
        node_data = {
            "Shortcode": json_data["node"]["shortcode"],
            "ID": json_data["node"]["id"],
            "__Typename": json_data["node"]["__typename"],
            "Date": json_data["node"].get("date"),  # Use .get() to handle missing key gracefully
            "Caption": json_data["node"]["caption"],
            "Like Counts": json_data["node"]["edge_media_preview_like"]["count"], 
            #"count": json_data["node"]["location"]["facebook_places_id"],

            "Comment_count": json_data["node"].get("edge_media_to_comment", {}).get("count"),
            "Iphone_struct": json_data["node"]["iphone_struct"]['location'],
           
            
            #"device_timestamp":json_data["device_timestamp"].get("device_timestamp")# Use .get() to handle missing key
        }
              # Extract location details from 'iphone_struct' ˇif it exists
        iphone_struct = json_data["node"]["iphone_struct"]
        location_info = iphone_struct.get("location")
        if location_info:
            node_data["Location_name"] = location_info.get("name")
            node_data["Location_address"] = location_info.get("address")
            node_data["Location_city"] = location_info.get("city")
            node_data["Location_pk"]= location_info.get("pk")
            node_data["Longtitude"] = location_info.get("lng")
            node_data["Latitude"] = location_info.get("lat")
            
        try:
            node_data["timestamp"] = pd.to_datetime(node_data["date"], unit="s").strftime('%Y-%m-%d %H:%M:%S')
        except (KeyError, ValueError):
            node_data["timestamp"] = np.nan  # Set timestamp to NaN
       
        return node_data
    except Exception as e:
        print(f"Error extracting fields from JSON data: {str(e)}")
        return None

def read_json_xz_to_dataframe(file_path):
    try:
        # Open the compressed JSON file
        with lzma.open(file_path, 'rb') as compressed_file:
            # Read and decode the JSON data
            json_data = json.loads(compressed_file.read().decode('utf-8'))
            
            # Extract the fields
            extracted_data = extract_fields_from_json(json_data)
            
            return extracted_data
    except Exception as e:
        print(f"Error reading JSON file: {str(e)}")
        return None

# List to store extracted data
extracted_data_list = []

# Iterate over folder paths
for folder_path in folder_paths:
    # Iterate over files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.json.xz'):
            file_path = os.path.join(folder_path, filename)
            extracted_data = read_json_xz_to_dataframe(file_path)
            if extracted_data is not None:
                extracted_data_list.append(extracted_data)

# Create a DataFrame from the extracted data
if extracted_data_list:
    df = pd.DataFrame(extracted_data_list)
else:
    print("No valid data found in the folders.")

# Assuming you want to change the name of the 'old_column_name' to 'new_column_name'
df.rename(columns={'location_pk': 'Location id'}, inplace=True)

print(len(df))
df

1844


,Shortcode,ID,__Typename,Date,Caption,Like Counts,Comment_count,Iphone_struct,Location_name,Location_address,Location_city,Location_pk,Longtitude,Latitude,timestamp
0,Cub5yNXxfV1,3142359304003515765,GraphSidecar,1688818441,Postkort fra et solrigt @amagerfaelledparkrun ...,47,None,"{'pk': 237843396, 'short_name': 'Amager Fælled...",Amager Fælled parkrun,Amager Fælled,Copenhagen,237843396,12.574479,55.652903,NaN
1,CjKev_trL3J,2939296946435440073,GraphImage,1664611522,"Blev mødt af friløbende legesyge hund, der hop...",19,None,"{'pk': 237843396, 'short_name': 'Amager Fælled...",Amager Fælled parkrun,Amager Fælled,Copenhagen,237843396,12.574479,55.652903,NaN
2,CnY-YUlLUSa,3015434300774237338,GraphImage,1673687802,Yay - back at #parkrun 🥳 I havn’t been for abo...,62,None,"{'pk': 237843396, 'short_name': 'Amager Fælled...",Amager Fælled parkrun,Amager Fælled,Copenhagen,237843396,12.574479,55.652903,NaN
3,CxPxqKXIQq0,3192989062451497652,GraphImage,1694853979,#parkrunamagerfælled fejrede løb #700 i dag. \...,11,None,"{'pk': 237843396, 'short_name': 'Amager Fælled...",Amager Fælled parkrun,Amager Fælled,Copenhagen,237843396,12.574479,55.652903,NaN
4,Cc953YWIyeG,2827670635422099334,GraphVideo,1651304704,"Thanks for the push, Heidi 🙏🏽😎🏃‍♂️\n\n#healthy...",35,None,"{'pk': 237843396, 'short_name': 'Amager Fælled...",Amager Fælled parkrun,Amager Fælled,Copenhagen,237843396,12.574479,55.652903,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1839,BmThz-EBng5,1842965379392370745,GraphImage,1533918615,Så er min kære bil og jeg på tur. Weekenden st...,22,None,"{'pk': 1815041615391306, 'short_name': 'Til he...",Til hest på Helgenæs v/ Rolf,Svanesmindevej 3,"Knebel, Arhus, Denmark",1815041615391306,10.553530,56.124990,NaN
1840,CH7Jofjn2uG,2448593187487312774,GraphSidecar,1606115076,I think we match ❤️\n.\n.\n.\n\n#picture #whit...,17,None,"{'pk': 1815041615391306, 'short_name': 'Til he...",Til hest på Helgenæs v/ Rolf,Svanesmindevej 3,"Knebel, Arhus, Denmark",1815041615391306,10.553530,56.124990,NaN
1841,Bw_Sz5wnv75,2035428288144277241,GraphImage,1556861981,Riding on the beach was amazing! .\n.\n.\n#bea...,43,None,"{'pk': 1815041615391306, 'short_name': 'Til he...",Til hest på Helgenæs v/ Rolf,Svanesmindevej 3,"Knebel, Arhus, Denmark",1815041615391306,10.553530,56.124990,NaN
1842,CDO5pQMFEjg,2364080377948293344,GraphImage,1596040364,Frihed ❤🐎,7,None,"{'pk': 1815041615391306, 'short_name': 'Til he...",Til hest på Helgenæs v/ Rolf,Svanesmindevej 3,"Knebel, Arhus, Denmark",1815041615391306,10.553530,56.124990,NaN


In [62]:
df.to_csv('df_instram_location_id_Amager_fælled', index = False)